# Analysis Summary

The goal from this analysis will be to predict the outcome of March Madness games. To do this, I will look at historical data from NCAA tourney matchups from 2003 up to 2018 output predictions for the winning team for each game. For every March Madness game, each team will be assigned a score based on its regular season performance in  several statistical categories, assigned rankings, and how well its oppponent performs in several statistical categories. 

### Baseline Models
I will be using a few baseline models to compare the model's results against. The baseline models I will be using are:

    - March Madness Seed: The seed assigned to each team for a March Madness tournament
    - Rating Percentage Index (RPI):  Ranking assigned to each team based on a team's wins, losses, and strength of record
    - Pomeroy College Basketball Ratings (POM): Ken Pomeroy's annual college basketball ratings 

The baseline models pose the following hypothesis:

    - That in any NCAA tournament game, the winner of a game will have either a lower March Madness seed, lower RPI, or lower POM rating.
  
Intuitively, this is a reasonable prediction. RPI, March Madness seeds, and POM ratings are all based on a team's wins, losses, strength of schedule, and generally considered representative of a team's performance relative to other teams. If team A has a lower RPI than team B at the end of a season, it's generally considered that team A's performance throughout the season has been at a higher level than team B's. For this reason, if we knew nothing else about the two teams, predicting the outcome based on RPI, or the other two benchmarks, is a good starting point. 

### Data Sources Overview
The data that was used to perform this analysis came from the [Google Cloud & NCAA® ML Competition 2018-Men's Kaggle competition](https://www.kaggle.com/c/mens-machine-learning-competition-2018). The following is a brief summary of all the datasets that were used:
- NCAATourneyCompactResults: Contains records from each NCAA tournament game from 1985-2017, including score and region.  
- Teams: Contains information for each Division 1 (D1) basketball team including an ID, name, first and most recent year playing D1 basketball. 
- MasseyOrdinals_Prelim2018: Contains data from 2003-2017 surrounding each D1 team's rank from various rankings sources throughout. 
- RegularSeasonDetailedResults: Contains similar information as the NCAATourneyCompactResults dataset, with the addition that each row will also contain the totals in a variety of statistical categories for the winning team and losing team. These are categories that are often found in a boxscore. 

A more in depth description of each of the datasets that were used and additional datasets provided by Kaggle can be found at https://www.kaggle.com/c/mens-machine-learning-competition-2018/data

### Data Cleansing & Preparation

In order for the model to be created, the following was completed:
    1. A data frame was created that holds a set of statistics for each team in a given year used to evaluate the that team's offensive and defensive performance. 
    2. A data frame was created, where each row represents a matchup that occurred in an NCAA tournament and contains the set of statistics for each team in the matchup 

To accomplish the first task, the following statistics were calculated for each team:
    - Three Point Percentage (3P%)
    - Two Point Percentage (2P%)
    - Defensive Rebounds (DR)
    - Offensive Rebounds (OR)
    - Field Goal Percentage (FG%)
    - Free Throws Attempted (FTA)
    - Free Throw Percentage (FTP)
    - Turnovers (TO)
    - Turnovers Forced (TOF)
    - Personal Fouls (PF)
    - Efficient Field Goal Percentage (eFG): (FGM + 0.5 * 3PM) / FGA
    - Possessions (POS): FGA - ORB + TO + (0.4 x FTA)
    - Defensive Efficiency (dEff): 100 x PA / (FGA - ORB + TO + (0.4 x FTA))   
    - Offensive Effiency (oEff): 100 x (PS) / (FGA - ORB + TO + (0.4 x FTA))
    - Three Point Usage (3PP): 3PA / FGA
    - Two Point Usage (2PP): 2PA / FGA
    - Rankings Percentage Index (RPI)
    - March Madness Seed (Seed)
    - Ken Pomeroy Rating (POM)

Each statistic was calcualated using game data leading up to the tournament, including conference play. RPI, March Madness Seed, and POM ratings were found using the latest ranking given to a team before the tournament.

An example of the output data frame is seen below with a snapshot of some teams' performance in 2003:

In [95]:
statsDemo = setupStatsDF(2003)
statsDemo.head()

,teamId,oEff,dEff,eFG,FG%,3P%,FT%,3PP,2P%,2PP,FTA,OR,DR,TO,TOF,POS,PF,season
0,1102_2003,1.062575,1.057935,0.678187,0.481149,0.375643,0.651357,0.409857,0.596987,0.590143,17.107143,4.178571,16.821429,11.428571,12.964286,53.878571,18.750000,2003
1,1103_2003,1.140972,1.131853,0.583886,0.486074,0.338710,0.736390,0.207334,0.545624,0.792666,25.851852,9.777778,19.925926,12.629630,15.333333,69.044444,19.851852,2003
2,1104_2003,1.061618,0.995951,0.531855,0.420362,0.320144,0.709898,0.275258,0.473684,0.724742,20.928571,13.571429,23.928571,13.285714,13.857143,65.264286,18.035714,2003
3,1105_2003,0.950489,1.015179,0.519039,0.395755,0.364815,0.705986,0.316720,0.411488,0.683280,21.846154,13.500000,23.115385,18.653846,18.807692,75.507692,20.230769,2003
4,1106_2003,0.954755,0.956899,0.534561,0.423773,0.346154,0.646421,0.288040,0.460152,0.711960,16.464286,12.285714,23.857143,17.035714,15.071429,66.621429,18.178571,2003


To complete the second requirement of preparing the data for the model, two steps were taken: 
1. The data frame that was created above and a data frame holding the rankings for all teams by year were merged based on a shared custom id. The custom id was a combination of a team's id and the season the corresponding row of data was associated to. 
2. The merged data frame's features were normalized, converting all values to a scale from 0 - 1. The features were normalized for two reasons: 
    1. To allow each record of data to be interpreted the same way
    2. Eliminate any additional influence a feature with a large range (FTA) would have compared to a feature with a smaller range of data (FT%). 

The output of these two steps can be seen below:


In [99]:
ranks = getRankings()
seeds = getSeeds()
seedsAndRanks = mergeSeedsAndRanks(seeds, ranks)
statsSeeds = mergeSeedsRanksStats(seedsAndRanks, statsDemo)
normStats = normalizeFeatures(statsSeeds)
normStats.head()

,teamId,oEff,dEff,eFG,FG%,3P%,FT%,3PP,2P%,2PP,...,OR,DR,TO,TOF,POS,PF,season,Seed,POM,RPI
0,1104_2003,0.240034,0.441460,0.081066,0.160056,0.024833,0.540265,0.517317,0.273052,0.482683,...,0.687804,0.467720,0.317239,0.400189,0.342766,0.409248,2003,0.6,0.117647,0.171296
1,1112_2003,0.669673,0.187895,0.386234,0.559339,0.347937,0.489395,0.395750,0.538274,0.604250,...,0.880254,0.957187,0.482932,0.757805,0.954239,0.372801,2003,0.0,0.007353,0.004630
2,1113_2003,0.613722,0.545460,0.221122,0.722442,0.000000,0.299035,0.000000,0.636862,1.000000,...,0.701961,0.386250,0.396140,0.598082,0.490229,0.585044,2003,0.6,0.113971,0.143519
3,1120_2003,0.188074,0.397887,0.422431,0.641261,0.288017,0.253537,0.366593,0.631687,0.633407,...,0.491648,0.288249,0.606018,0.631788,0.421546,0.196334,2003,0.6,0.161765,0.162037
4,1122_2003,0.179950,0.544580,0.379396,0.471159,0.333096,0.428905,0.442242,0.487484,0.557758,...,0.335534,0.506417,0.531149,0.417219,0.403711,0.489736,2003,0.8,0.448529,0.333333


### Model Creation & Score Assignment

Using the normalized statistics data frame and a data frame consisting of historical records of each March Madness tournament matchup, a score can be calculated for each team to predict the outcome of a tournament game. Each team is given a score based on several criteria:

#### Offensive Performance Score
$$ OffScore_A = oEff_A*(1 - dEff_B) + 4(3P\%_A) + 8(2P\%_A) + 5FTA_A + 3.5FT\%_A + 2OR_A - 2.5TO_A $$
A team's offensive performance score is based on several features:
- Offensive Efficiency: $oEff_A*(1 - dEff_B)$: 
    - Using team A's offensive efficiency gives a good estimate of how many points per 100 possessions a team scores. From that value, a team is only granted a percentage of it based on how well team B plays defense. 
- Shooting: $4(3P\%_A) + 8(2P\%_A) + 5FTA_A + 3.5FT\%_A$
    - A team is rewarded for how well they shoot three pointers, two pointers, and free throws, as well as how many free throws they shoot a game. 
- Rebounding & Turnovers: $2OR_A - 2.5TO_A$: 
    - Team A is rewarded if they tend to be better at offensive rebounding than other teams and are penalized for how often they turn the ball over. 
    
#### Defensive Performance Score:
$$ DefScore_A = 8.5(1-dEff_A) + 5DR_A + 3TOF_A - 5PF_A $$
Team A is rewarded based on their defensive efficiency, how well they rebound defensively, how often they force turnovers, and penalized for how often they foul. 
#### External Ranking Score: 
$$ ExtRankScore_A = 12.5(1 - Seed_A) + 10(1 - POM_A) + 7.5(1 - RPI_A) $$ 
A team is given an external ranking score based on their RPI before the tournament, their March Madness seed, and where they stand in Ken Pomeroy's end of the season rankings. The lower the ranking or seeding, the higher the reward. Coefficients for each metric were based on how well each one independently predicted March Madness games.

#### Cumulative Score
The sum of all three scores are taken to assign each team a final score:
$$ FinScore = OffScore + DefScore + ExtRankScore $$

Once scores are calculated for each team in each matchup, a prediction for the outcome of a game is made by selecting the team with a higher score as the winner. A final data frame is created showing the scores calculated for each team, how  my model performed predicting the outcome of the game, and how each baseline model performed predicting the outcome of the game. A snippet of that data frame can be seen below:

In [265]:
matchDemo = createModelMatchupsDF()
matchDemo.head()

,season,wId,wName,lId,lName,wScore,lScore,wRPI,wPOM,wSeed,lRPI,lPOM,lSeed,predicted,seedBaseline,pomBaseline,rpiBaseline
0,2003,1112_2003,Arizona,1436_2003,Vermont,54.053825,22.698741,0.004630,0.007353,0.000000,0.662037,0.525735,1.000000,True,True,True,True
1,2003,1113_2003,Arizona St,1272_2003,Memphis,36.137226,40.579217,0.143519,0.113971,0.600000,0.115741,0.069853,0.400000,False,False,False,False
2,2003,1141_2003,C Michigan,1166_2003,Creighton,37.822274,46.738843,0.199074,0.180147,0.666667,0.101852,0.095588,0.333333,False,False,False,False
3,2003,1143_2003,California,1301_2003,NC State,37.760308,35.052224,0.148148,0.143382,0.466667,0.240741,0.172794,0.533333,True,True,True,True
4,2003,1163_2003,Connecticut,1140_2003,BYU,44.474833,37.141720,0.125000,0.080882,0.266667,0.083333,0.091912,0.733333,True,True,True,False


### Predictive Model and Baseline Model Comparison

How my model compares against each of the baseline models can be seen in the data frame below:

In [267]:
modelComparisons = new.groupby('season')["predicted", "seedBaseline", "pomBaseline", "rpiBaseline"].agg('sum')
modelComparisons

,predicted,seedBaseline,pomBaseline,rpiBaseline
season,,,,
2003,46.0,42.0,43.0,44.0
2004,47.0,47.0,46.0,47.0
2005,48.0,43.0,47.0,46.0
2006,46.0,42.0,45.0,40.0
2007,51.0,50.0,49.0,46.0
2008,49.0,47.0,51.0,46.0
2009,47.0,47.0,44.0,46.0
2010,45.0,42.0,45.0,44.0
2011,40.0,43.0,43.0,39.0


Out of the 15 years that predictions were made for:
- My model performed better than or as well as all baseline models in 10/15 years
- My model performed worse than the POM baseline model in only 3/15 years
- My model performed worse than the Seed baseline model in only 4/15 years
- My model performed worse than the RPI baseline model in only 1/15 years

#### Significance Testing
In order to determine if the difference in number of games predicted by my model and each baseline model is significant, a one-tailed paired sample t-test was performed with a significance level of .05. The null and alternate hypothesis for this test will be as follows: 
$$ H_0: \mu_i - \mu_j = 0 $$
$$ H_1: \mu_i - \mu_j > 0 $$

where $\mu_i$ represents the mean number of games predicted correctly by my model over 15 seasons and $\mu_j$ represents the mean number of games predicted correctly by a selected baseline model over 15 seasons.

For this test, a significance level of .05 will be used to determine whether or not the difference in number of games predicted correctly between the two models is significant. The results of the three t-tests can be seen below:

In [299]:
testModForSig(modelComparisons)

,seedBaseline,pomBaseline,rpiBaseline
0,0.043547,0.107508,0.001158


For two of the three baseline tests (Seed & RPI), the paired-samples t-test above results in a p-value that is less than our significance level of .05, showing that the differences in accurately predicted games is large enough to reject the null hypothesis that this difference is not significant. 


### Future Steps & Potential Improvements
- Random Forest to determine which features most important 
- Determine why when having offensive efficiency * (1 - team A POS - team B POS) everything works out

In [74]:
import pandas as pd 
from scipy import stats as sciStats
import numpy as np
import team, game as g
import Model as mod

"""
stats for each team found from a certain year 
seeds and rankings are found for each team from a certain year 
data frames are merged linking stats to seeds/ranks
merged data frame is normalized to give all stats or ranks equal weight

each matchup from a year's tournament is used to calculate a score for each team
score is calculated using the normalized features and the logic in the findScore function
- Offensive score, defensive score, ranking score that are combined 
- weights used were based on four factors of basketball and then adjusted 

score for both team is calculated and then used to create a new data frame for the corresponding matchup 
a score for both teams in each matchup is calculated and a data frame with those scores is created 
A prediction for each matchup in data frame is found by looking at which team has highest calculated score 

model predictions are compared against baseline models based on how many games were picked accurately by year 
one tailed proportions test was done to test whether or not the models results were significant 
"""

In [75]:
def getTeamNames():
    """
    Return dictionary where key is team ID and value is team name
    """
    names = {}
    teams = pd.read_csv("Data/Teams.csv")
    for index, row in teams.iterrows():
        teamId = row["TeamID"]
        name = row["TeamName"]
        names[teamId] = name
    return names

def getSeasonStats(ncaaTourneyTeams):
    """
    Use regular season results and RPI rankings to create a 
    dictionary where key is the team's ID and the value is a 
    Team object. Team objects contain yearly avg stats for each 
    team in various categories.
    """
    teams = {}
    names = getTeamNames()
    unfiltRanks = pd.read_csv("data/MasseyOrdinals_Prelim2018.csv")
    rankings = unfiltRanks[(unfiltRanks["SystemName"] == "RPI") & (unfiltRanks["RankingDayNum"] == 133)]
    regSeasonResults = pd.read_csv("data/RegularSeasonDetailedResults.csv")
    for index, row in regSeasonResults.iterrows():
        season = row["Season"]
        dayNum = row["DayNum"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)
        wRPI = None
        lRPI = None
        try:
            wRPI = rankings[(rankings["Season"] == season) & (rankings["TeamID"] == wTeamId)].iloc[0]["OrdinalRank"]
            lRPI = rankings[(rankings["Season"] == season) & (rankings["TeamID"] == lTeamId)].iloc[0]["OrdinalRank"]
        except Exception as e:
            pass
            # print str(lTeamId) + " " + str(season) + " not found"
        
        if customWId not in teams:
            teams[customWId] = team.Team(customWId)
        if customLId not in teams:
            teams[customLId] = team.Team(customLId)
        wTeam = teams[customWId]
        wTeam.RPI = wRPI
        wTeam.name = names[wTeamId]
        wTeam.updateStats(row, True)
        if customLId in ncaaTourneyTeams:
            wTeam.winsVsTourney += 1
        lTeam = teams[customLId]
        lTeam.name = names[lTeamId]
        lTeam.RPI = lRPI
        lTeam.updateStats(row, False)
        
    for t in teams:
        teams[t].setScore()
    
    return teams

def getTeamStats(teams):
    """
    Get season stats for each team in a Data Frame. Able to see yearly averages and totals for stored
    statistical categories in each team object in teams dictionary
    """
    allTeamData = []
    visited = set()
    for team in teams:
        if team not in visited:
            idAndSeason = team.split("_")
            season = idAndSeason[1]
            _id = idAndSeason[0]
            teamData = teams[team].objToDict()
            teamData['season'] = season
            allTeamData.append(teamData)
            visited.add(team)
    return pd.DataFrame(allTeamData)

def dataToDict(df):
    """
    Convert dataframe to list to dictionary where key is team id and value is row data
    """
    data = {}
    for index, row in df.iterrows():
        data[row["_id"]] = row.to_dict()
    return data

def normalizeFeatures(teamStats):
    """
    Uses team stats for all seasons and returns data frame with stats normalized by season
    """
    toReturn = teamStats.copy()
    copy = toReturn.copy()
    copy['season'] = copy['season'].astype(int)
    copy.drop(copy.select_dtypes(['object']), inplace=True, axis=1)
    normalized = normalize(copy, 'season')
#     normalized.drop(columns=['numGamesPlayed'])
    toReturn.update(normalized)
    return toReturn

def normalize(df, by):
    """
    groups df by season and normalizes each statistical category of features from 0 - 1
    """
    groups = df.groupby(by)
    mins = groups.transform(np.min)
    maxs = groups.transform(np.max)
    return (df[mins.columns] - mins) / (maxs - mins)

def populateNCAATourneyTeams():
    """
    Create an ID for each team using a combination of its id and the season the team played in. 
    Output a dictionary with an entry for each team whose key is its newly created id
    """
    ncaaTourneyTeams = {}
    ncaaTournResults = pd.read_csv("data/NCAATourneyCompactResults.csv")
    for index, row in ncaaTournResults.iterrows():
        season = row["Season"]
        dayNum = row["DayNum"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)

        if customWId not in ncaaTourneyTeams:
            ncaaTourneyTeams[customWId] = 1
        if customLId not in ncaaTourneyTeams:
            ncaaTourneyTeams[customLId] = 1
    return ncaaTourneyTeams

def getMatchups(teams, inputDataModified=False):
    """
    Use NCAA Tournament results to return data frame of matchups where each row contains data for one matchup between two teams, including their yearly avg totals in statistical categories, RPI, and game result.
    """
    matchups = []
    ncaaTournResults = pd.read_csv("data/NCAATourneyCompactResults.csv")
    for index, row in ncaaTournResults.iterrows():
        season = row["Season"]
        dayNum = row["DayNum"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)

        if customWId in teams and customLId in teams:
            wTeamData, lTeamData = {}, {}
            if inputDataModified:
                wTeamData = teams[customWId].copy()
                del wTeamData['season']
            else:
                wTeamData = teams[customWId].objToDict().copy()
            wTeamDataMod, lTeamDataMod = {}, {}
            for key in wTeamData.keys():
                wTeamDataMod["w" + key] = wTeamData[key]
                
            if inputDataModified:
                lTeamData = teams[customLId]
                del lTeamData['season']
            else:
                lTeamData = teams[customLId].objToDict().copy()
            for key in lTeamData.keys():
                lTeamDataMod["l" + key] = lTeamData[key]
                
            matchupData = wTeamDataMod.copy()
            matchupData.update(lTeamDataMod)
            matchupData["dayNum"] = dayNum
            matchupData["season"] = season
            matchups.append(matchupData)
    colOrder = ["dayNum", "season", "l_id", "lname", "w_id", "wname", "lscore", "wscore", "lDRB", "lEFG", "lFTA", "lFTP", "lMOL", "lMOV", "lORB", "lPOSS",
                "lRPI", "lTO", "lTOF", "lconfTournWins", "ldEff", "lnumGamesPlayed", "loEff", "lwinsVsTourney",
                "wDRB", "wEFG", "wFTA", "wFTP", "wMOL", "wMOV", "wORB", "wPOSS", "wRPI", "wTO", "wTOF", 
                "wconfTournWins", "wdEff", "wnumGamesPlayed", "woEff", "wwinsVsTourney"]
    df = pd.DataFrame.from_dict(matchups)
    df = df[colOrder]
    return df

def getMatchupData():
    """
    Returns data frame of historical matchups in NCAA tournament.
    Reads in existing CSV if available. Otherwise, produces data frame by creating Team objects, calculating yearly avg totals for each team, and joining with historical NCAA tourney matchup data
    """
    try:
        matchups = pd.read_csv("Data/output/matchups_normalized.csv")
        return matchups
    except Exception as e:
        ncaaTourneyTeams = populateNCAATourneyTeams()
        teamObjs = getSeasonStats(ncaaTourneyTeams)
        teamStats = getTeamStats(teamObjs)
        normalized = normalizeFeatures(teamStats)
        dataAsDict = dataToDict(normalized)
        matchups = getMatchups(dataAsDict, inputDataModified=True)
        matchups.to_csv("Data/output/matchups_normalized.csv", index=False)
        return matchups

In [264]:
def getRankings():
    unfiltRanks = pd.read_csv("data/MasseyOrdinals_Prelim2018.csv")
    rankings = unfiltRanks[(unfiltRanks["SystemName"] == "POM") & (unfiltRanks["RankingDayNum"] == 133) | (unfiltRanks["SystemName"] == "RPI") & (unfiltRanks["RankingDayNum"] == 133)]
    transpose = {}
    for index, row in rankings.iterrows():
        teamId = str(row["TeamID"]) + "_" + str(row["Season"])
        if teamId in transpose:
            if row["SystemName"] == "RPI":
                transpose[teamId]["RPI"] = row["OrdinalRank"]
            else:
                transpose[teamId]["POM"] = row["OrdinalRank"]
        else:
            transpose[teamId] = {}
            if row["SystemName"] == "RPI":
                transpose[teamId]["RPI"] = row["OrdinalRank"]
            else:
                transpose[teamId]["POM"] = row["OrdinalRank"]
    ranksDF = pd.DataFrame.from_dict(orient = "index", data = transpose).reset_index()
    ranksDF.columns = ["teamId", "POM", "RPI"]
    return ranksDF

def getSeeds():
    seeds = pd.read_csv("data/NCAATourneySeeds.csv")
    seeds["teamId"] = seeds.TeamID.astype(str).str.cat(seeds.Season.astype(str), sep='_')
    seeds["Seed"] = seeds.Seed.str.extract('(\d+)', expand=False).astype(int)
    seeds = seeds.drop(["Season", "TeamID"], axis = 1)
    return seeds

def mergeSeedsAndRanks(seeds, ranks):
    return seeds.merge(ranks, how = "inner", on = "teamId")

def mergeSeedsRanksStats(seedsAndRanks, stats):
    return stats.merge(seedsAndRanks, how = "inner", on = "teamId")

def normalizeFeatures(teamStats):
    """
    Uses team stats for all seasons and returns data frame with stats normalized by season
    """
    toReturn = teamStats.copy()
    copy = toReturn.copy()
    copy['season'] = copy['season'].astype(int)
    copy.drop(copy.select_dtypes(['object']), inplace=True, axis=1)
    normalized = normalize(copy, 'season')
    toReturn.update(normalized)
    return toReturn

def normalize(df, by):
    """
    groups df by season and normalizes each statistical category of features from 0 - 1
    """
    groups = df.groupby(by)
    mins = groups.transform(np.min)
    maxs = groups.transform(np.max)
    return (df[mins.columns] - mins) / (maxs - mins)

def findScore(teamA, teamB):
    offScoreData = [teamA["oEff"].values[0],2*teamA["3PP"].values[0],3*teamA["3P%"].values[0],8*teamA["FG%"].values[0],5*teamA["FTA"].values[0],3.5*teamA["FT%"].values[0],2*teamA["OR"].values[0],2.5*teamA["TO"].values[0]]
    defScoreData = [9*(1-teamA["dEff"]).values[0], 5*teamA["DR"].values[0], 3*teamA["TOF"].values[0], 5*teamA["PF"].values[0]]
    seedData = [12.5*(1-teamA["Seed"]).values[0], 10*(1-teamA["POM"]).values[0], 7.5*(1-teamA["RPI"]).values[0]]
    offScore = (teamA["oEff"]*(1 - teamB["dEff"]).values[0] + 4*teamA["3P%"] + 8*teamA["2P%"] + 5*teamA["FTA"] + 3.5*teamA["FT%"] + 2*teamA["OR"] - 2.5*teamA["TO"])
    defScore = (8.5*(1-teamA["dEff"]) + 5*teamA["DR"] + 3*teamA["TOF"] - 5*teamA["PF"])
    seedScore = 12.5*(1-teamA["Seed"]) + 10*(1-teamA["POM"]) + 7.5*(1-teamA["RPI"])
    total = offScore.values[0] + defScore.values[0] + seedScore.values[0]
    scoreData = [teamA["teamId"].values[0]] + offScoreData + [offScore.values[0]] + defScoreData + [defScore.values[0]] + seedData + [seedScore.values[0]] + [total]
    return scoreData


In [218]:
def setupStatsDF(year):
    allGames = []
    regSeasonResults = pd.read_csv("data/RegularSeasonDetailedResults.csv")
    games = regSeasonResults[regSeasonResults["Season"] == year]
    for index, row in games.iterrows():
        season = row["Season"]
        dayNum = row["DayNum"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)
        row1 = [customWId, season, dayNum, row["WScore"], row["LScore"], row[30]] + list(row[8:21])
        row2 = [customLId, season, dayNum, row["LScore"], row["WScore"], row[17]] + list(row[21:])
        allGames.append(row1)
        allGames.append(row2)
    colNames = ["teamId", "season", "dayNum", "score", "oppScore", "TOF", "FGM", "FGA", "FGM3", "FGA3", "FTM", "FTA", "OR", "DR", "Ast", 
               "TO", "Stl", "Blk", "PF"]
    gamesDF = pd.DataFrame(allGames, columns = colNames)
    gamesDF = gamesDF.drop(["dayNum"], axis = 1)
    gamesDF["season"] = gamesDF["season"].astype("object")
    gamesDF["teamId"] = gamesDF["teamId"].astype("object")
    statDevs = gamesDF.groupby(by = "teamId").std().reset_index() ## Consider using later
    gamesDF["POS"] = gamesDF["FGA"] - gamesDF["OR"] + gamesDF["TO"] + (0.4 * gamesDF["FTA"])
    sums = gamesDF.groupby(by = "teamId").sum().reset_index()
    means = gamesDF.groupby(by = "teamId").mean().reset_index()

    sums["oEff"] = sums["score"]/sums["POS"]
    sums["dEff"] = sums["oppScore"]/sums["POS"]
    sums["eFG"] = (sums["FGM"] + (0.5 + sums["FGM3"])) / sums["FGA"]
    sums["FG%"] = sums["FGM"]/sums["FGA"]
    sums["3P%"] = sums["FGM3"]/sums["FGA3"]
    sums["3PP"] = 3*sums["FGM3"]/sums["score"]
    sums["FT%"] = sums["FTM"]/sums["FTA"]
    sums["2P%"] = (sums["FGM"] - sums["FGM3"]) / (sums["FGA"] - sums["FGA3"])
    sums["2PP"] = 1 - sums["3PP"]

    meanSub = means[["teamId", "FTA", "OR", "DR", "TO", "TOF", "POS", "PF"]]
    sumSub = sums[["teamId", "oEff", "dEff", "eFG", "FG%", "3P%", "FT%", "3PP", "2P%", "2PP"]]
    stats = sumSub.merge(meanSub, how='left', left_on = "teamId", right_on = "teamId")
    stats["season"] = stats["teamId"].apply(lambda x: x[x.index("_") + 1:])
    return stats


In [298]:
def createModelMatchupsDF():
    ranks = getRankings()
    seeds = getSeeds()
    seedsAndRanks = mergeSeedsAndRanks(seeds, ranks)
    names = getTeamNames()
    matchups = []
    teamScoreInfo = []
    visited = set()
    for year in range(2003, 2018):
        statsDF = setupStatsDF(year)
        statsSeeds = mergeSeedsRanksStats(seedsAndRanks, statsDF)
        normStats = normalizeFeatures(statsSeeds)
        tournRes = pd.read_csv("data/NCAATourneyCompactResults.csv")
        tournRes = tournRes[tournRes["DayNum"] > 135]
        recentRes = tournRes[tournRes["Season"] == year]

        for index, row in recentRes.iterrows():
            season = row["Season"]
            wTeamId = row["WTeamID"]
            lTeamId = row["LTeamID"]
            customWId = str(wTeamId) + "_" + str(season)
            customLId = str(lTeamId) + "_" + str(season)
            teamW = normStats[normStats["teamId"] == customWId]
            teamL = normStats[normStats["teamId"] == customLId]
            seedRankW = [teamW["RPI"].values[0], teamW["POM"].values[0], teamW["Seed"].values[0]]
            seedRankL = [teamL["RPI"].values[0], teamL["POM"].values[0], teamL["Seed"].values[0]]
            if not teamW.empty:
                wScore = findScore(teamW, teamL)
                lScore = findScore(teamL, teamW)
                if customWId not in visited:
                    teamScoreInfo.append(wScore)
                    visited.add(customWId)
                if customLId not in visited:
                    teamScoreInfo.append(lScore)
                    visited.add(customLId)
                matchups.append([season, customWId, names[wTeamId], customLId, names[lTeamId], wScore[-1], lScore[-1]] + seedRankW + seedRankL)
    matchDF = pd.DataFrame(matchups, columns = ["season", "wId", "wName", "lId", "lName", "wScore", "lScore", 
                                               "wRPI", "wPOM", "wSeed", "lRPI", "lPOM", "lSeed"])
    matchDF["predicted"] = matchDF["wScore"] > matchDF["lScore"]
    matchDF["seedBaseline"] = matchDF["wSeed"] < matchDF["lSeed"]
    matchDF["pomBaseline"] = matchDF["wPOM"] < matchDF["lPOM"]
    matchDF["rpiBaseline"] = matchDF["wRPI"] < matchDF["lRPI"]
    matchDF.to_csv("data/output/matchupRes_regression.csv", index = False)
    return matchDF

def testModForSig(modelComparisons):
    baselines = ["seedBaseline", "pomBaseline", "rpiBaseline"]
    np.random.seed(12345678)
    results = {}
    for base in baselines:
        myModel = np.array(modelComparisons["predicted"])
        seed = np.array(modelComparisons[base])
        results[base] = sciStats.ttest_rel(myModel,seed).pvalue/2
    resDF = pd.DataFrame(data = results, index=range(1))
    return resDF
                                  

In [255]:
# no less than for seedbaseline
modelComparisons = matchDF.groupby('season')["predicted", "seedBaseline", "pomBaseline", "rpiBaseline"].agg('sum')

In [256]:
myModel = np.array(modelComparisons ["predicted"])
seed = np.array(modelComparisons["seedBaseline"])
np.random.seed(12345678)
sciStats.ttest_rel(myModel,seed)

Ttest_relResult(statistic=0.9715041043243726, pvalue=0.3477840191501982)

In [257]:
res.head()

,season,wId,wName,lId,lName,wScore,lScore,wRPI,wPOM,wSeed,lRPI,lPOM,lSeed,predicted,seedBaseline,pomBaseline,rpiBaseline
0,2003,1112_2003,Arizona,1436_2003,Vermont,54.147573,22.929009,0.004630,0.007353,0.000000,0.662037,0.525735,1.000000,True,True,True,True
1,2003,1113_2003,Arizona St,1272_2003,Memphis,36.558784,40.860170,0.143519,0.113971,0.600000,0.115741,0.069853,0.400000,False,False,False,False
2,2003,1141_2003,C Michigan,1166_2003,Creighton,38.149458,47.851408,0.199074,0.180147,0.666667,0.101852,0.095588,0.333333,False,False,False,False
3,2003,1143_2003,California,1301_2003,NC State,37.927440,35.497580,0.148148,0.143382,0.466667,0.240741,0.172794,0.533333,True,True,True,True
4,2003,1163_2003,Connecticut,1140_2003,BYU,44.520137,38.055825,0.125000,0.080882,0.266667,0.083333,0.091912,0.733333,True,True,True,False


In [258]:
new = createModelMatchupsDF()
new.head()

1    12.063839
dtype: float64
57    6.21126
dtype: float64
2    11.18945
dtype: float64
30    8.218155
dtype: float64
7    16.223044
dtype: float64
13    13.568919
dtype: float64
8    8.905728
dtype: float64
34    11.562609
dtype: float64
12    10.92469
dtype: float64
6    13.29859
dtype: float64
15    13.032842
dtype: float64
11    12.896434
dtype: float64
18    13.264885
dtype: float64
9    5.771903
dtype: float64
20    12.240046
dtype: float64
58    9.652315
dtype: float64
23    11.147599
dtype: float64
56    8.765184
dtype: float64
28    15.135011
dtype: float64
19    9.835115
dtype: float64
33    6.874271
dtype: float64
43    10.409695
dtype: float64
35    11.819553
dtype: float64
62    11.46842
dtype: float64
36    9.122171
dtype: float64
42    9.087204
dtype: float64
47    8.001536
dtype: float64
45    11.312715
dtype: float64
51    9.53236
dtype: float64
14    11.870784
dtype: float64
63    11.008247
dtype: float64
61    13.392683
dtype: float64
3    8.105037
dtype: float64
46 

64    7.689244
dtype: float64
34    10.823508
dtype: float64
2    11.350172
dtype: float64
50    10.318386
dtype: float64
18    13.481622
dtype: float64
30    8.609051
dtype: float64
22    10.233037
dtype: float64
7    11.437834
dtype: float64
49    11.691487
dtype: float64
17    16.025146
dtype: float64
54    13.506154
dtype: float64
37    11.528495
dtype: float64
60    13.316896
dtype: float64
40    13.666621
dtype: float64
61    10.014079
dtype: float64
59    15.272093
dtype: float64
62    9.713981
dtype: float64
3    10.242555
dtype: float64
11    12.076491
dtype: float64
27    8.845418
dtype: float64
23    13.546946
dtype: float64
16    9.007603
dtype: float64
25    15.04481
dtype: float64
57    9.889008
dtype: float64
29    11.966519
dtype: float64
8    10.992534
dtype: float64
33    15.932065
dtype: float64
20    6.482161
dtype: float64
38    14.307002
dtype: float64
43    7.306133
dtype: float64
58    10.898943
dtype: float64
14    13.610641
dtype: float64
64    7.890405
dtype:

28    10.42487
dtype: float64
49    13.905009
dtype: float64
37    10.378412
dtype: float64
47    8.454647
dtype: float64
51    9.040268
dtype: float64
42    9.375686
dtype: float64
12    15.493278
dtype: float64
5    10.590248
dtype: float64
15    12.866315
dtype: float64
55    9.366294
dtype: float64
34    13.873618
dtype: float64
53    9.103259
dtype: float64
40    9.867565
dtype: float64
52    8.674852
dtype: float64
37    10.873221
dtype: float64
28    10.329345
dtype: float64
51    9.398518
dtype: float64
22    10.897431
dtype: float64
12    15.391721
dtype: float64
40    10.14301
dtype: float64
15    13.361467
dtype: float64
34    13.871847
dtype: float64
12    15.637043
dtype: float64
51    9.250468
dtype: float64
37    10.578402
dtype: float64
15    13.346976
dtype: float64
12    15.681468
dtype: float64
37    10.711514
dtype: float64
16    14.408095
dtype: float64
5    11.782297
dtype: float64
22    15.96064
dtype: float64
40    12.502421
dtype: float64
23    10.67693
dtype: 

1    12.483342
dtype: float64
42    12.441777
dtype: float64
2    12.395737
dtype: float64
54    10.976029
dtype: float64
3    14.422037
dtype: float64
9    9.438778
dtype: float64
15    13.932426
dtype: float64
18    10.755639
dtype: float64
16    12.214956
dtype: float64
30    11.625698
dtype: float64
17    12.916633
dtype: float64
8    7.188672
dtype: float64
27    13.431878
dtype: float64
55    13.086987
dtype: float64
28    11.375379
dtype: float64
25    10.969057
dtype: float64
31    8.749521
dtype: float64
52    10.394043
dtype: float64
34    9.113181
dtype: float64
11    11.966447
dtype: float64
37    6.894286
dtype: float64
32    13.446661
dtype: float64
46    13.402517
dtype: float64
40    8.198325
dtype: float64
49    8.990895
dtype: float64
43    9.983962
dtype: float64
57    13.939828
dtype: float64
41    7.751639
dtype: float64
58    7.832547
dtype: float64
50    11.06872
dtype: float64
59    11.560137
dtype: float64
20    10.592042
dtype: float64
4    12.569085
dtype: fl

55    10.607375
dtype: float64
52    9.637116
dtype: float64
31    12.76077
dtype: float64
67    8.378715
dtype: float64
43    8.388179
dtype: float64
1    11.222731
dtype: float64
6    7.446639
dtype: float64
18    14.921622
dtype: float64
59    6.490304
dtype: float64
23    12.929547
dtype: float64
20    9.589723
dtype: float64
28    6.322554
dtype: float64
39    10.483299
dtype: float64
30    10.806717
dtype: float64
37    12.664944
dtype: float64
45    11.273247
dtype: float64
16    12.594879
dtype: float64
54    10.433488
dtype: float64
22    8.350749
dtype: float64
66    7.574466
dtype: float64
60    10.508229
dtype: float64
5    5.18374
dtype: float64
14    7.906942
dtype: float64
13    11.509821
dtype: float64
41    7.479064
dtype: float64
21    11.72707
dtype: float64
46    8.44289
dtype: float64
33    10.859994
dtype: float64
52    9.72033
dtype: float64
38    9.63457
dtype: float64
15    9.091073
dtype: float64
42    10.682019
dtype: float64
9    15.38941
dtype: float64
44  

1    13.469172
dtype: float64
53    7.8574
dtype: float64
2    9.782265
dtype: float64
65    7.539875
dtype: float64
7    7.566496
dtype: float64
52    6.984492
dtype: float64
9    5.634311
dtype: float64
46    8.688381
dtype: float64
15    9.413064
dtype: float64
14    11.713421
dtype: float64
17    9.836204
dtype: float64
3    6.939002
dtype: float64
25    12.437457
dtype: float64
19    4.024423
dtype: float64
34    7.169022
dtype: float64
28    6.255887
dtype: float64
36    9.441454
dtype: float64
20    5.11896
dtype: float64
40    15.848468
dtype: float64
38    10.575583
dtype: float64
41    10.671374
dtype: float64
58    6.575183
dtype: float64
54    5.72324
dtype: float64
23    11.933983
dtype: float64
56    6.842104
dtype: float64
50    10.554319
dtype: float64
57    12.270895
dtype: float64
49    14.688601
dtype: float64
60    13.150682
dtype: float64
26    11.561843
dtype: float64
67    10.523204
dtype: float64
32    8.602786
dtype: float64
12    10.656513
dtype: float64
45   

14    10.285426
dtype: float64
44    8.874972
dtype: float64
6    11.103972
dtype: float64
48    5.937957
dtype: float64
22    13.102773
dtype: float64
53    14.707594
dtype: float64
19    7.629147
dtype: float64
55    11.12845
dtype: float64
47    11.234387
dtype: float64
64    13.590743
dtype: float64
7    11.35328
dtype: float64
0    12.90973
dtype: float64
49    12.434258
dtype: float64
4    12.510453
dtype: float64
29    10.037059
dtype: float64
11    11.144324
dtype: float64
60    7.68767
dtype: float64
13    15.703069
dtype: float64
37    8.312285
dtype: float64
43    12.116141
dtype: float64
15    9.765212
dtype: float64
63    10.807545
dtype: float64
38    11.857914
dtype: float64
66    7.804894
dtype: float64
59    15.982199
dtype: float64
67    10.118936
dtype: float64
12    11.996472
dtype: float64
2    9.380212
dtype: float64
55    11.05831
dtype: float64
17    11.777068
dtype: float64
26    7.755915
dtype: float64
20    13.519134
dtype: float64
64    13.622027
dtype: floa

,season,wId,wName,lId,lName,wScore,lScore,wRPI,wPOM,wSeed,lRPI,lPOM,lSeed,predicted,seedBaseline,pomBaseline,rpiBaseline
0,2003,1112_2003,Arizona,1436_2003,Vermont,54.053825,22.698741,0.004630,0.007353,0.000000,0.662037,0.525735,1.000000,True,True,True,True
1,2003,1113_2003,Arizona St,1272_2003,Memphis,36.137226,40.579217,0.143519,0.113971,0.600000,0.115741,0.069853,0.400000,False,False,False,False
2,2003,1141_2003,C Michigan,1166_2003,Creighton,37.822274,46.738843,0.199074,0.180147,0.666667,0.101852,0.095588,0.333333,False,False,False,False
3,2003,1143_2003,California,1301_2003,NC State,37.760308,35.052224,0.148148,0.143382,0.466667,0.240741,0.172794,0.533333,True,True,True,True
4,2003,1163_2003,Connecticut,1140_2003,BYU,44.474833,37.141720,0.125000,0.080882,0.266667,0.083333,0.091912,0.733333,True,True,True,False


In [259]:
modelComparisons2 = new.groupby('season')["predicted", "seedBaseline", "pomBaseline", "rpiBaseline"].agg('sum')

In [260]:
myModel = np.array(modelComparisons2["predicted"])
seed = np.array(modelComparisons2["seedBaseline"])
np.random.seed(12345678)
sciStats.ttest_rel(myModel,seed)

Ttest_relResult(statistic=1.8398011906324638, pvalue=0.08709442666232664)

In [261]:
modelComparisons2

,predicted,seedBaseline,pomBaseline,rpiBaseline
season,,,,
2003,46.0,42.0,43.0,44.0
2004,47.0,47.0,46.0,47.0
2005,48.0,43.0,47.0,46.0
2006,46.0,42.0,45.0,40.0
2007,51.0,50.0,49.0,46.0
2008,49.0,47.0,51.0,46.0
2009,47.0,47.0,44.0,46.0
2010,45.0,42.0,45.0,44.0
2011,40.0,43.0,43.0,39.0


In [262]:
matchDF.head()

,season,wId,wName,lId,lName,wScore,lScore,wRPI,wPOM,wSeed,lRPI,lPOM,lSeed,predicted,seedBaseline,pomBaseline,rpiBaseline
0,2003,1112_2003,Arizona,1436_2003,Vermont,55.920368,22.613380,0.004630,0.007353,0.000000,0.662037,0.525735,1.000000,True,True,True,True
1,2003,1113_2003,Arizona St,1272_2003,Memphis,37.037016,41.342357,0.143519,0.113971,0.600000,0.115741,0.069853,0.400000,False,False,False,False
2,2003,1141_2003,C Michigan,1166_2003,Creighton,38.922240,47.923103,0.199074,0.180147,0.666667,0.101852,0.095588,0.333333,False,False,False,False
3,2003,1143_2003,California,1301_2003,NC State,38.569433,35.734119,0.148148,0.143382,0.466667,0.240741,0.172794,0.533333,True,True,True,True
4,2003,1163_2003,Connecticut,1140_2003,BYU,46.196861,37.585121,0.125000,0.080882,0.266667,0.083333,0.091912,0.733333,True,True,True,False
